# AIBECS beta-test

Welcome to the beta test of the AIBECS!
Well, we are not sure about the name yet, but as of now, AIBECS stands for the **Algebraic Implicit Biogeochemistry Elemental-Cycling System**.
AIBECS may be pronounced like the cool [ibex](https://en.wikipedia.org/wiki/Ibex) (if you have a french accent).
AIBECS is a new software written in [Julia](https://julialang.org) to easily create some marine biogeochmistry models in just a few commands.

*AIBECS is developed primarily by Benoît Pasquier (pasquieb@uci.edu) with the help of François Primeau (fprimeau@uci.edu) and J. Keith Moore (jkmoore@uci.edu).*

**Wait, I'm confused... What is AIBECS exactly?**

AIBECS is not just a single model.
It's a **system** that allows you to create a global steady-state biogeochmistry model with just a few simple commands.
Basically, you just need to tell AIBECS
- (i) which ocean circulation to use (from simple toy models of just a few boxes to more complicated global models of the circulation),
- (ii) what elements you want to model/track and how each tracer gets converted into other tracers, and
- (iii) chose some model parameters to start with.
Once the model is set up, you can run simulations.

**"Algebraic", "implicit", what do these mean? I thought I was doing some oceanography today...**

Well, AIBECS relies on many tools from linear algebra to run simulations and perform optimizations really fast.
AIBECS-generated models are described by a state function, denoted $\boldsymbol{F}$, which defines how the concnetrations of elements in the ocean evolve with time.
In mathematical terms, this translates to a system of nonlinear differential equations with the generic form 

$$\frac{\partial \boldsymbol{x}}{\partial t} = \boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p}),$$

where $\boldsymbol{x}$ is the state of the model (i.e., the concentrations of the tracers), and $\boldsymbol{p}$ are model parameters.
Here, we are interested in the equilibrium of the system (AKA the steady-state).
That is when the time-derivative is $0$, so that

$$\boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p}) = 0,$$

and $\boldsymbol{x}$ does not change with time.
Instead of simulating the evolution of $\boldsymbol{x}$ with time and waiting for the system to reach equilibrium — like most biogeochemistry models do — AIBECS uses linear algebra techniques, like Newton's method in multiple dimensions, or Krylov spaces, to implicitly solve for the steady-state solution, hence the "algebraic" and "implicit" names.
This makes AIBECS much faster than the competition!
Note again that we are not set on the name of the software yet, so if you have a good idea, don't hesitate to share it with us!

## OK, enought words, show me some AIBECS action!

First things first, we must tell Julia that we want to use the package:

In [1]:
using TransportMatrixTools

┌ Info: Precompiling TransportMatrixTools [3487739c-c12e-11e8-1d55-2958ce19ee2a]
└ @ Base loading.jl:1186
Expr(:const, Symbol("##bin_index_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be broken for this module **

ERROR: LoadError: CodecZlib.jl is not installed properly, run Pkg.build("CodecZlib") and restart Julia.
Stacktrace:
 [1] error(::String) at ./error.jl:33
 [2] top-level scope at /Users/lorencalle/.julia/packages/CodecZlib/9jDi1/src/CodecZlib.jl:34
 [3] include at ./boot.jl:326 [inlined]
 [4] include_relative(::Module, ::String) at ./loading.jl:1038
 [5] include(::Module, ::String) at ./sysimg.jl:29
 [6] top-level scope at none:2
 [7] eval at ./boot.jl:328 [inlined]
 [8] eval(::Expr) at ./client.jl:404
 [9] top-level scope at ./none:3
in expression starting at /Users/lorencalle/.julia/packages/CodecZlib/9jDi1/src/CodecZlib.jl:33
ERROR: LoadError: LoadError: Failed to precompile CodecZlib [944b1d66-785c-5afd-91f1-9de20f533193] to /Users/lorencalle/.

ErrorException: Failed to precompile TransportMatrixTools [3487739c-c12e-11e8-1d55-2958ce19ee2a] to /Users/lorencalle/.julia/compiled/v1.1/TransportMatrixTools/6EUXl.ji.

If it's the first time you are running this line, the package will be precompiled. 
This may take a minute or two.
Just be patient :)
(You might also see some warnings, just disregard those... This is)

### Set up a model

Let's start with a very simple model.
We will simulate the mean age of water.
In theory, the mean age is average amount of time spent by a water parcel since its last contact with the surface.
Here the "surface" will be the top layer of the model grid.
So, we have one tracer, the age, which we will denote by `age` in Julia.
But before diving into the equations, let us chose the circulation. 

#### The circulation

We will use the circulation output from the Ocean Circulation Inverse Model (OCIM) version 1.1.
Basically, the OCIM provides scientists, like you and me, with a big sparse matrix that represents the global ocean circulation (advection and diffusion).
(For more details, see Tim DeVries's [website](https://tdevries.eri.ucsb.edu/models-and-data-products/) and references therein.)
With AIBECS, the OCIM circulation can be loaded really easily, by simply typing 

In [ ]:
const wet3d, grd, T_OCIM = TransportMatrixTools.OCIM1.load() ;

(We use the `const` prefix to tell Julia these are constants — it makes Julia faster!)
Julia may ask you to download the OCIM matrix for you, in which case you should say `yes`.
Once downloaded, TransportMatrixTools will remember where it downloaded the file and it will only load it from your laptop.
Additionally to downloading the OCIM file, the last command loads 3 variables in the Julia workspace:
- `grd` is a dictionary (a `Dict` in Julia, equivalent to a `struct` in MATLAB) containing variables and extra information about the 3D grid of the circulation, like latitudes, longitudes, and depths of grid boxes.
- `wet3d` is a 3D array of the model grid, filled with `1`'s at "wet" grid boxes and `0`'s and "land" grid boxes.
- `T_OCIM` is the transport matrix representing advection and diffusion.

Let's check that `T_OCIM` is what we think it is, by using the `typeof` command:

In [ ]:
typeof(T_OCIM)

Great, it's a sparse matrix! (CSC just means that it is stored in Compressed Sparse Column format.) 

We can get its size with the `size` command:

In [ ]:
size(T_OCIM)

Prefect! Let's move on with setting up the model!

So the state of the model, $\boldsymbol{x}$, is entirely defined by one tracer: the age. 
So we have only one tracer, `age`. The age is transported along with water parcels, so its transport matrix is the ocean circulation matrix that we just loaded: `T_OCIM`.
We tell AIBECS that by first writing the transport matrix for the age as a function of the parameters (although there are no parameters in this simple model).

In [ ]:
T_age(p) = T_OCIM

(Functions in Julia can be created in one line, just as above.)

#### The local sources and sinks

Now what are the local sources and sinks of `age`?
The age increases by $1$ second every, well, second, everywhere.
So its source is $1$ everywhere.
The age is also $0$ at the surface.
To implement that we use a fast-relaxation technique, which acts as a sink for the age.
That is, we enforce a strong restoration of the age to `0` in the top layer of the model.
AIBECS can generate a number of useful constants for you, like the vector of depths, `z`. This is done via

In [ ]:
const nb, DIV, Iabove, v, z, ztop = constants(wet3d, grd) ;

These are:

- `nb` — the number of wet grid boxes. Here, this is also the length of the state vector `x`, because there is only one tracer, `age`.
- `DIV` and `Iabove` — not useful for now, but for your information they are the discrete spatial divergence operator, and a matrix shifting indices to grid boxes above, respectively.
- `v` — the vector of grid box volumnes.
- `z` — the vector of grid box depths.
- `ztop` — the vector of the depths of the top of the grid boxes.

We can now create the source for the age (which is `1` everywhere), via

In [ ]:
source_age(age, p) = 1

For the sink, we must first define what the top layer is.
Let's investigate what's the minimum depth:

In [ ]:
minimum(z)

So the surface layer in the OCIM grid, which is regular (or flat, if you prefer), has its center at about 18m.
Hence, we can create a mask of the surface layer via `z < 20`, for example.
Then the local sink can be implemented with a fast relaxation, via

In [ ]:
function sink_age(age, p)
    τ = p.τ
    return age .* (z < 20) / τ
end

(Julia allows you to use unicode for your functions and variables, like for `τ`.)
Here, we have defined a Julia function using the `function` keyword because the sink is a bit more complicated, so that we needed two lines to define it.
The first line unpacks the model parameters.
Here these are just `τ`, the relaxation timescale, which we will set to a low value later, to ensure that the retoring to `0` in the surface layer is fast. 

Now the sources minus the sinks is simply defined by

In [ ]:
sms_age(age, p) = source_age(age, p) .- sink_age(age, p)

Finally, the last step for the set up is to define $\boldsymbol{F}$.
Using AIBECS, this is done via

In [5]:
nt = 1                           # number of tracers
T_matrices = (T_OCIM,)           # bundles all the transport matrices in a tuple
sources_minus_sinks = (sms_age,) # bundles all the source-sink functions in a tuple
F, ∇ₓF = state_function_and_Jacobian(Ts, Gs, nt, nb) # generates the state function (and its Jacobian!)

UndefVarError: UndefVarError: state_function_and_Jacobian not defined

Here we have just created a model of the mean age.
The first 3 lines in the cell above are just telling AIBECS
- how many tracers there are,
- what transport matrices it should use for the transport of these tracers
- and what local sources and sinks should be appplied to these tracers
(Note that this interface was developed for multiple tracers, and might look like too much here, and I might come around to simplify this part in the case of a single tracer, but as of now, this is what we have to do!)

The last line creates two functions, `F`, which is the numerical version of $\boldsymbol{F}$ of our model of the mean age, and `∇ₓF`, which is the Jacobian matrix of the state function, i.e., $\nabla_{\boldsymbol{x}}\boldsymbol{F}$.
Yes, AIBECS just automatically created an exact derivative of your input, using autodifferentiation via dual numbers.
(But this is an entirely different discussion.)

The Jacobian, `∇ₓF` is essential to solving the steady-state equation $\boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p}) = 0$ fast.
This is done via Newton's method, by starting from an initial guess, that you will have to provide, it will iterate via the recursion
$$\boldsymbol{x}_{k+1} = \boldsymbol{x}_{k} - \nabla_{\boldsymbol{x}}\boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p})^{-1} \boldsymbol{F}(\boldsymbol{x}, \boldsymbol{p}) $$

Now I am nice, so I wrote up an algorithm and an API to solve for the steady-state.
Before we do so, we must first chose some parameters... And I made an API for that!



In [ ]:
t = empty_parameter_table()    # initialize table of parameters
add_parameter!(t, :τ, 1u"s")   # add the parameter we want (τ = 1s)
initialize_parameter_type(t)   # Generate the parameter type

The lines above created a parameter vector, `p₀`. 
Let us have a look at it

In [ ]:
p₀

Here I did not really need to create `p₀` as a parameters vector.
But we are here to learn, and this structure and functionality comes in very handy when one deals with many parameters.
Anyway, now lets finish our model, and set an initial guess for the age.
Let's assume the age is `1` (seconds) evrywhere:

In [ ]:
x₀ = ones(nb)

### Run the simulation, i.e., solve for the steady state

First, we create an instance of the steady-state problem, via

In [ ]:
prob = SteadyStateProblem(F, ∇ₓF, x₀, p₀)

Finally, we can find the solution via the `solve` function:

In [ ]:
age = solve(prob, CTKAlg(); preprint=" ", maxItNewton=50)

That's it! 
We solved for the steady state in just one line!
Everyone here deserves a nice tap on the shoulder — Good job!
Now let's see what this age looks like on a map

### Make some figures

